In [1]:
import speech_recognition as sr
import torch
import numpy as np
from transformers import BertTokenizer
import pandas as pd
from torch import nn
from transformers import BertModel

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [3]:
model_dict_path = './models/model_bert.pth'
model_dict = torch.load(model_dict_path)

In [4]:
model = BertClassifier()
model.load_state_dict(model_dict['model_state_dict'])
model.eval()

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [5]:
text = "誰說的"

In [9]:
def predict(model, text):
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    unlabeled_input = tokenizer(text, padding='max_length', max_length = 32, truncation=True, return_tensors="pt")

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    unlabeled_mask = unlabeled_input['attention_mask'].to(device)
    unlabeled_input_id = unlabeled_input['input_ids'].squeeze(1).to(device)

    unlabeled_output = model(unlabeled_input_id, unlabeled_mask)
    print(unlabeled_output)
    print(unlabeled_output.argmax(dim=1))
    pseudo_label = unlabeled_output.argmax(dim=1).item()
    print(text, pseudo_label)
    return pseudo_label

In [10]:
offensive_predict = predict(model, text)

tensor([[3.3154, 8.2724, 0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)
tensor([1])
誰說的 1


In [14]:
text_df = pd.DataFrame([[text, offensive_predict]], columns=['text', 'label'])

In [15]:
labelsMap = {0:'Offensive',
             1:'Non-offensive'
            }

text_df.label = text_df.label.map(labelsMap)

In [16]:
text_df

,text,label
0,誰說的,Non-offensive


In [ ]:
r = sr.Recognizer()

In [ ]:
with sr.Microphone() as source:
    audio_data = r.record(source, duration = 10)
    print("Recognizing...")
    text = r.recognize_google(audio_data, language="zh-TW")
    print(text)